# 第三章 LangChain进阶组件实操

#### 3.1.2.1 全量记忆

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

# 加载环境变量（确保.env文件中配置了API_KEY）
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

# 初始化LLM模型
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3  # 降低随机性，保证输出稳定
)

# 1. 定义提示词模板（包含历史消息占位符）
full_memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是友好的对话助手，需基于完整的历史对话回答用户问题。"),
    MessagesPlaceholder(variable_name="chat_history"),  # 历史消息占位符
    ("human", "{user_input}")  # 用户当前输入
])

# 2. 构建基础链（提示词 + LLM）
base_chain = full_memory_prompt | llm

# 3. 会话历史存储（内存模式，生产环境可替换为数据库存储）
full_memory_store = {}

# 4. 定义会话历史获取函数（核心：返回完整历史）
def get_full_memory_history(session_id: str) -> BaseChatMessageHistory:
    """根据session_id获取会话历史，不存在则创建新的历史记录"""
    if session_id not in full_memory_store:
        full_memory_store[session_id] = InMemoryChatMessageHistory()
    return full_memory_store[session_id]

# 5. 构建带全量记忆的对话链
full_memory_chain = RunnableWithMessageHistory(
    runnable=base_chain,
    get_session_history=get_full_memory_history,
    input_messages_key="user_input",  # 输入中用户问题的键名
    history_messages_key="chat_history"  # 传入提示词的历史消息键名
)

# 测试多轮对话（指定session_id=user_001，隔离不同用户）
config = {"configurable": {"session_id": "user_001"}}

# 第一轮对话
response1 = full_memory_chain.invoke({"user_input": "我叫小明，喜欢编程"}, config=config)
print("助手回复1：", response1.content)
# 输出示例：你好小明！编程是一项很有创造力的技能，你平时常用什么编程语言呢？

# 第二轮对话（验证记忆：询问历史信息）
response2 = full_memory_chain.invoke({"user_input": "我刚才说我喜欢什么？"}, config=config)
print("助手回复2：", response2.content)

# 查看完整历史记录
print("\n全量记忆的对话历史：")
for msg in get_full_memory_history("user_001").messages:
    print(f"{msg.type}: {msg.content}")

e:\anaconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


助手回复1： 你好小明！很高兴认识你！编程是个非常棒的爱好，能创造、解决问题，还能实现各种有趣的想法。你主要对哪种编程语言或领域感兴趣呢？比如网页开发、数据分析、游戏设计，还是其他方向？ 😊
助手回复2： 你刚才提到你喜欢编程！需要我推荐一些学习资源、项目灵感，或者聊聊编程相关的话题吗？ 😄

全量记忆的对话历史：
human: 我叫小明，喜欢编程
ai: 你好小明！很高兴认识你！编程是个非常棒的爱好，能创造、解决问题，还能实现各种有趣的想法。你主要对哪种编程语言或领域感兴趣呢？比如网页开发、数据分析、游戏设计，还是其他方向？ 😊
human: 我刚才说我喜欢什么？
ai: 你刚才提到你喜欢编程！需要我推荐一些学习资源、项目灵感，或者聊聊编程相关的话题吗？ 😄


#### 3.1.2.2 窗口记忆

In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

# 加载环境变量（确保.env文件中配置了API_KEY）
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

# 初始化LLM模型
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3  # 降低随机性，保证输出稳定
)

# 1. 定义提示词模板（与全量记忆通用，可复用）
window_memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是友好的对话助手，需基于最近的对话历史回答用户问题。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# 2. 构建基础链
window_base_chain = window_memory_prompt | llm

# 3. 会话历史存储
window_memory_store = {}
WINDOW_SIZE = 2  # 保留最近2轮对话（即最近4条消息：用户-助手-用户-助手）

# 4. 定义带窗口限制的会话历史获取函数
def get_window_memory_history(session_id: str) -> BaseChatMessageHistory:
    """获取会话历史，仅保留最近WINDOW_SIZE轮对话"""
    if session_id not in window_memory_store:
        window_memory_store[session_id] = InMemoryChatMessageHistory()
    
    # 获取完整历史，截取最近WINDOW_SIZE轮（每轮2条消息）
    history = window_memory_store[session_id]
    if len(history.messages) > 2 * WINDOW_SIZE:
        # 截取后WINDOW_SIZE轮消息（保留最新的）
        history.messages = history.messages[-2 * WINDOW_SIZE:]
    return history

# 5. 构建带窗口记忆的对话链
window_memory_chain = RunnableWithMessageHistory(
    runnable=window_base_chain,
    get_session_history=get_window_memory_history,
    input_messages_key="user_input",
    history_messages_key="chat_history"
)

# 测试多轮对话（session_id=user_002，与全量记忆会话隔离）
config = {"configurable": {"session_id": "user_002"}}

# 模拟5轮对话，验证窗口记忆的截断效果
inputs = [
    "我叫小红",
    "我喜欢画画",
    "我来自上海",
    "我是一名学生",
    "我刚才说我来自哪里？"  # 第5轮：询问第3轮的信息，验证窗口截断
]

for i, user_input in enumerate(inputs, 1):
    response = window_memory_chain.invoke({"user_input": user_input}, config=config)
    print(f"\n第{i}轮 - 助手回复：", response.content)

# 查看窗口记忆的最终历史（仅保留最近2轮）
print("\n窗口记忆的最终对话历史（最近2轮）：")
for msg in get_window_memory_history("user_002").messages:
    print(f"{msg.type}: {msg.content}")


第1轮 - 助手回复： 你好小红！很高兴认识你！有什么我可以帮助你的吗？

第2轮 - 助手回复： 画画是很棒的爱好呢！你通常喜欢画什么类型的作品呢？比如风景、人物，还是抽象画？

第3轮 - 助手回复： 上海是个充满艺术气息的城市呢！外滩的夜景、石库门的老建筑，还有各种美术馆和创意园区，应该给你带来不少创作灵感吧？你平时会画一些上海的城市风景吗？

第4轮 - 助手回复： 学生时期能有时间坚持画画真的很珍贵呢！你是艺术专业的学生，还是把画画当作课余爱好？如果方便的话，可以分享一下最近在画的作品吗？

第5轮 - 助手回复： 你刚才提到你来自**上海**。这座城市既有现代都市的活力，又保留着丰富的历史文化痕迹，确实很适合作为艺术创作的灵感来源呢！

窗口记忆的最终对话历史（最近2轮）：
human: 我是一名学生
ai: 学生时期能有时间坚持画画真的很珍贵呢！你是艺术专业的学生，还是把画画当作课余爱好？如果方便的话，可以分享一下最近在画的作品吗？
human: 我刚才说我来自哪里？
ai: 你刚才提到你来自**上海**。这座城市既有现代都市的活力，又保留着丰富的历史文化痕迹，确实很适合作为艺术创作的灵感来源呢！


#### 3.1.2.3 摘要记忆

In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

# 加载环境变量（确保.env文件中配置了API_KEY）
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

# 初始化LLM模型
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3  # 降低随机性，保证输出稳定
)

# 1. 定义摘要生成提示词（用于压缩对话历史）
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是对话摘要助手，需简洁总结以下对话的核心信息（包含用户身份、偏好、关键问题等），不超过50字。"),
    ("human", "对话历史：{chat_history_text}\n请生成摘要：")
])

# 2. 构建摘要生成链（输入完整历史文本，输出摘要）
summary_chain = summary_prompt | llm

# 3. 定义对话记忆提示词（注入摘要而非完整历史）
summary_memory_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是友好的对话助手，需基于对话摘要回答用户问题，摘要包含核心上下文信息。"),
    ("system", "对话摘要：{chat_summary}"),  # 注入摘要
    ("human", "{user_input}")
])

# 4. 构建基础对话链（提示词 + LLM）
summary_base_chain = (
    RunnablePassthrough.assign(
        chat_summary=lambda x: summary_chain.invoke(
            {
                "chat_history_text": "\n".join(
                    [f"{msg.type}: {msg.content}" for msg in x["chat_history"]]
                )
            }
        ).content
    )
    | summary_memory_prompt
    | llm
)

# 5. 会话历史存储（保存完整历史用于生成摘要）
summary_memory_store = {}

# 6. 定义会话历史获取函数
def get_summary_memory_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in summary_memory_store:
        summary_memory_store[session_id] = InMemoryChatMessageHistory()
    return summary_memory_store[session_id]

# 7. 构建带摘要记忆的对话链
summary_memory_chain = RunnableWithMessageHistory(
    runnable=summary_base_chain,
    get_session_history=get_summary_memory_history,
    input_messages_key="user_input",
    history_messages_key="chat_history"  # 传入完整历史用于生成摘要
)

# 测试多轮对话（session_id=user_003）
config = {"configurable": {"session_id": "user_003"}}

# 多轮对话输入
inputs = [
    "我叫小李，是一名产品经理",
    "我负责一款电商APP的迭代",
    "最近在优化用户下单流程",
    "遇到了用户流失率高的问题",
    "你能给我一些优化建议吗？"
]

for i, user_input in enumerate(inputs, 1):
    response = summary_memory_chain.invoke({"user_input": user_input}, config=config)
    print(f"\n第{i}轮 - 助手回复：", response.content)

# 查看完整历史与最终摘要
history = get_summary_memory_history("user_003")
print("\n摘要记忆的完整对话历史：")
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

# 单独生成最终摘要验证
final_summary = summary_chain.invoke({
    "chat_history_text": "\n".join([f"{msg.type}: {msg.content}" for msg in history.messages])
}).content
print(f"\n最终对话摘要：{final_summary}")


第1轮 - 助手回复： 你好小李！很高兴认识你！作为产品经理，你的工作一定充满挑战和创意吧？如果需要讨论产品设计、用户需求或任何相关话题，我随时可以帮忙哦！ 😊

第2轮 - 助手回复： 很高兴能为您提供支持！作为产品经理，您对电商APP的迭代有什么具体想法或需要讨论的方向吗？比如用户需求分析、功能优化，还是用户体验设计？

第3轮 - 助手回复： 好的，小李。优化下单流程是提升转化率和用户体验的关键。基于我们之前的讨论，我们可以从几个核心方向切入：

1.  **简化与提速**：这是最直接的优化点。我们可以分析当前流程的步骤数、必填字段和页面跳转。目标是让核心用户（如已登录、有默认地址）能在**3步内、15秒内**完成下单。具体可以：
    *   **减少非必要信息**：例如，对大部分商品隐藏“发票信息”填写，改为可选项。
    *   **智能填充与预测**：地址自动联想，根据历史购买记录预选支付方式。
    *   **流程合并**：将“确认订单”和“支付”页面的信息再次精简，尝试“一键下单”模式（在商品页直接结算）。

2.  **清晰与信任**：在简化同时，必须确保关键信息透明，消除用户疑虑。
    *   **费用明细实时汇总**：商品价、运费、优惠券抵扣、实付金额必须清晰、实时计算并突出显示。
    *   **库存与时效强化提示**：对紧俏商品，在订单页明显提示库存数量；预估送达时间需要更醒目。
    *   **信任符号**：在支付按钮附近展示安全支付、隐私保护等图标或简短文案。

3.  **容错与引导**：优化不仅要为顺利流程设计，也要为中断和犹豫设计。
    *   **随时可返回修改**：用户能方便地返回修改地址、优惠券或商品数量，而无需取消整个订单。
    *   **异常状态友好提示**：如商品价格变动、库存售罄，提示语要清晰并给出明确替代方案（如“该商品已售罄，查看相似商品”）。
    *   **促进决策**：在用户停留时间较长的环节（如选择优惠券时），可以加入“XX%的用户选择了XX支付”等轻量引导。

**为了精准推进，我建议我们下一步：**

*   **数据诊断**：我们先拉取最近一个月的下单转化漏斗数据，定位流失最高的具体步骤。
*   **用户调研**：对在关键步骤流失的用户进行抽样访谈，了解他们的

#### 3.2.1.2 学习案例：查天气

In [12]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# ======================
# 1. 环境
# ======================
load_dotenv()
API_KEY = os.getenv("API_KEY")

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com",
    model="deepseek-chat",
    temperature=0.3,
)

# ======================
# 2. 工具
# ======================
@tool
def weather_query(city: str) -> str:
    """查询指定城市天气"""
    weather_data = {
        "北京": "北京今日天气：晴，-2~8℃",
        "上海": "上海今日天气：多云，5~12℃",
        "广州": "广州今日天气：小雨，18~25℃",
    }
    return weather_data.get(city, f"暂无 {city} 数据")

tools = [weather_query]

# ======================
# 3. 创建 Agent（开启 debug）
# ======================
agent = create_agent(
    model=llm,
    tools=tools,
    debug=True,  # 👈 打开过程打印
)

# ======================
# 4. 运行
# ======================
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "北京今天的天气怎么样？"}
    ]
})

print("\n最终回答：")
print(response["messages"][-1].content)


[values] {'messages': [HumanMessage(content='北京今天的天气怎么样？', additional_kwargs={}, response_metadata={}, id='0db9a6fc-3c72-4fcf-ad68-6a15d900806b')]}
[updates] {'model': {'messages': [AIMessage(content='我来帮您查询北京今天的天气情况。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 300, 'total_tokens': 351, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 44}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'd36afcda-ff41-4696-84e0-e3cbbb7cc1a5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4ab8-efb2-7af3-ab05-01086480cc91-0', tool_calls=[{'name': 'weather_query', 'args': {'city': '北京'}, 'id': 'call_00_AfoX4Qmlpu9zShh9rG2rCp4i', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 300, 'output_tokens

#### 3.2.2.2 学习案例:温度单位转换

In [14]:
from langchain.agents import create_agent # 最新 1.x 推荐入口
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

# ======================
# 1. 环境变量
# ======================
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3,
)

# ======================
# 2. 参数模型
# ======================
class TemperatureConvertInput(BaseModel):
    temperature: float = Field(description="需要转换的温度值，例如37.0")
    from_unit: str = Field(description="原始温度单位，只能是'celsius'（摄氏度）或'fahrenheit'（华氏度）")

# ======================
# 3. 自定义工具
# ======================
@tool(args_schema=TemperatureConvertInput)
def temperature_converter(temperature: float, from_unit: str) -> str:
    """温度单位转换工具"""
    if from_unit not in ["celsius", "fahrenheit"]:
        return f"错误：单位'{from_unit}'不合法，仅支持'celsius'或'fahrenheit'"
    
    if from_unit == "celsius":
        fahrenheit = temperature * 9/5 + 32
        return f"{temperature}摄氏度 = {fahrenheit:.2f}华氏度"
    else:
        celsius = (temperature - 32) * 5/9
        return f"{temperature}华氏度 = {celsius:.2f}摄氏度"

tools = [temperature_converter]

# ======================
# 4. Prompt模板（保留 agent_scratchpad）
# ======================
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一名专业温度转换助手，仅使用TemperatureConverter工具完成任务。"
     "严格校验参数，工具返回结果原样输出。"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # 记录思考和工具调用
])

# ======================
# 5. 创建 Agent
# ======================
agent = create_agent(
    model=llm,
    tools=tools,
    debug=True,  # 👈 打开调试，显示模型调用过程
)

# ======================
# 6. 执行任务
# ======================
if __name__ == "__main__":
    # 示例输入
    query = "将37摄氏度转换为华氏度"
    
    # 调用 agent
    response = agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })

    # 输出最终回答
    print("\n最终结果：")
    print(response["messages"][-1].content)


[values] {'messages': [HumanMessage(content='将37摄氏度转换为华氏度', additional_kwargs={}, response_metadata={}, id='fc5356d0-28b6-4392-b665-5730bd9b7300')]}
[updates] {'model': {'messages': [AIMessage(content='我来帮您将37摄氏度转换为华氏度。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 358, 'total_tokens': 430, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 166}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'c4af102e-828e-43f3-ac46-35ac06ec3a28', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4abb-9922-7463-ab82-ff91418b5c8d-0', tool_calls=[{'name': 'temperature_converter', 'args': {'temperature': 37, 'from_unit': 'celsius'}, 'id': 'call_00_qnglxQECidoucaLpGWLbZaei', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metad

#### 3.2.3.1 学习案例：创建文件

In [18]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import FileManagementToolkit
from dotenv import load_dotenv
import os

# -------------------
# 1. 初始化环境
# -------------------
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3,
)

# -------------------
# 2. 创建文件管理工具
# -------------------
toolkit = FileManagementToolkit(root_dir=".")
tools = toolkit.get_tools()

# -------------------
# 3. 创建 Agent（最新版）
# -------------------
agent = create_agent(
    model=llm,
    tools=tools,
    debug=True,  # 打开调试，显示模型思考和工具调用过程
)

# -------------------
# 4. 执行任务
# -------------------
response = agent.invoke({
    "messages": [
        {"role": "user", "content": "请创建一个名为 llm诗词.txt 的文件，并在文件中写入一首原创七言绝句，主题围绕科技与人文的融合。"}
    ]
})

print("\n任务执行完成！文件已写入。")
print("Agent最终输出：\n", response["messages"][-1].content)


[values] {'messages': [HumanMessage(content='请创建一个名为 llm诗词.txt 的文件，并在文件中写入一首原创七言绝句，主题围绕科技与人文的融合。', additional_kwargs={}, response_metadata={}, id='5d74eb6e-8bd9-4b5e-a155-5ac3f551f15e')]}
[updates] {'model': {'messages': [AIMessage(content='我将为您创建一个名为"llm诗词.txt"的文件，并写入一首原创的七言绝句。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 823, 'total_tokens': 936, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 631}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '647c09cf-f6ec-4e6a-8119-d4985bdee6c3', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c4ac4-5068-7741-b9e3-e04e61e70fb8-0', tool_calls=[{'name': 'write_file', 'args': {'file_path': 'llm诗词.txt', 'text': '《智联古今》\n硅基算力织云锦，\n代码如诗绘古今。\n人文科技相辉映，\n智能时代共知音。'}, 'id': 'c

### 3.3.1 实践1：带记忆的对话机器人

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
import re
import os
from dotenv import load_dotenv

# -------------------------
# 1. 初始化环境与模型
# -------------------------
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

calc_tool = PythonREPLTool()  # 数学计算工具
WINDOW_SIZE = 2  # 窗口记忆轮数

# -------------------------
# 2. 提示模板
# -------------------------
prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
你是一名友好的个人助手，能记住最近{WINDOW_SIZE}轮对话。
如果用户问题包含数学计算，请先调用计算工具，再生成自然语言回答。
"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# -------------------------
# 3. 工具调用逻辑
# -------------------------
def judge_and_calc(inputs):
    user_input = inputs["input"]
    
    calc_pattern = r"(\+|\-|\*|/|×|÷|=|计算|求和|求差|平方|立方|多少|等于)"
    if re.search(calc_pattern, user_input):
        # 提取表达式（简化版）
        expr = re.sub(r"[^\d\+\-\*\/\(\)\.]", "", user_input)
        calc_result = calc_tool.run(expr) if expr else "未识别到可计算的表达式"
        inputs["input"] = f"用户问题：{user_input}\n计算结果：{calc_result}\n请结合计算结果回答"
    return inputs

# -------------------------
# 4. 窗口记忆管理
# -------------------------
window_memory_store = {}

def get_window_session_history(session_id: str):
    if session_id not in window_memory_store:
        window_memory_store[session_id] = InMemoryChatMessageHistory()
    history = window_memory_store[session_id]
    # 保留最近N轮
    total = len(history.messages)
    if total > 2 * WINDOW_SIZE:
        history.messages = history.messages[-2 * WINDOW_SIZE:]
    return history

# -------------------------
# 5. 构建链
# -------------------------
chain = (
    RunnablePassthrough.assign(chat_history=lambda x: x.get("chat_history", []))
    | RunnableLambda(judge_and_calc)
    | prompt
    | llm
)

chain_with_memory = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_window_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output"
)

# -------------------------
# 6. 预设对话列表（模拟用户输入）
# -------------------------
predefined_dialogues = [
    "你好，我今天想问一下37摄氏度是多少华氏度？",
    "如果把212华氏度转换成摄氏度是多少？",
    "帮我算一下 15 + 28 × 2",
    "谢谢你，今天的温度真热啊"
]

# -------------------------
# 7. 模拟多轮对话执行
# -------------------------
if __name__ == "__main__":
    session_id = "student_001"
    print("===== 窗口记忆数学助手（预设对话） =====\n")
    
    for idx, user_input in enumerate(predefined_dialogues, 1):
        print(f"用户：{user_input}")
        
        response = chain_with_memory.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        
        print(f"助手：{response.content}\n")


===== 窗口记忆数学助手（预设对话） =====

用户：你好，我今天想问一下37摄氏度是多少华氏度？
助手：37摄氏度换算成华氏度是 **98.6华氏度**。  
如果你有其他温度需要换算，或者有其他问题，随时告诉我！

用户：如果把212华氏度转换成摄氏度是多少？
助手：212华氏度换算成摄氏度是 **100摄氏度**。  
这个温度正好是水的沸点（在标准大气压下），所以它是个很常见的参考值！有其他问题可以继续问我哦。

用户：帮我算一下 15 + 28 × 2
助手：根据数学运算顺序（先乘除后加减），计算如下：

\[
15 + 28 \times 2 = 15 + 56 = 71
\]

所以结果是 **71**。  
有其他需要计算的题目，随时告诉我！

用户：谢谢你，今天的温度真热啊
助手：不客气！确实，最近天气越来越热了，记得多喝水、注意防暑降温哦～ 🌞  
如果还有其他问题或需要帮忙的地方，随时告诉我！



### 3.3.2 实践2：带记忆的文件夹操作助手

In [ ]:
# =========================
# 1. 基础依赖
# =========================
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_agent

from langchain_core.messages import AIMessage, ToolMessage

# =========================
# 2. 模型
# =========================
load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url="https://api.deepseek.com",
    model="deepseek-chat",
    temperature=0.3
)

# =========================
# 3. 记忆
# =========================
WINDOW_SIZE = 3
memory_store = {}

def get_session_history(session_id: str):
    if session_id not in memory_store:
        memory_store[session_id] = InMemoryChatMessageHistory()

    history = memory_store[session_id]

    if len(history.messages) > 2 * WINDOW_SIZE:
        history.messages = history.messages[-2 * WINDOW_SIZE:]

    return history

# =========================
# 4. 工具
# =========================
@tool
def list_files(path: str = ".") -> str:
    """查看指定目录下的文件列表"""
    if not os.path.exists(path):
        return f"路径不存在：{path}"

    items = os.listdir(path)
    if not items:
        return "目录为空"

    result = []
    for item in items:
        full = os.path.join(path, item)
        if os.path.isfile(full):
            result.append(f"文件：{item}（{os.path.getsize(full)} 字节）")
        else:
            result.append(f"文件夹：{item}")

    return "\n".join(result)


@tool
def create_file(path: str, content: str = "") -> str:
    """创建文件并写入内容"""
    folder = os.path.dirname(path)
    if folder and not os.path.exists(folder):
        os.makedirs(folder)

    with open(path, "w", encoding="utf-8") as f:
        f.write(content)

    return f"文件已创建：{path}"

tools = [list_files, create_file]

# =========================
# 5. 创建 Agent
# =========================
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="你是一个文件操作智能助手。"
)

agent_with_memory = RunnableWithMessageHistory(
    agent,
    get_session_history,
    input_messages_key="messages",
    history_messages_key="messages"
)

# =========================
# 6. CLI
# =========================
if __name__ == "__main__":
    session_id = "tool_agent_demo"

    print("===== 🧠 LangChain v1.2 Agent =====")

    while True:
        user_input = input("你：")

        if user_input in ["q", "quit", "退出"]:
            print("助手：再见 👋")
            break

        # ✅ 打印用户输入
        print("\n👤【用户输入】")
        print(user_input)

        result = agent_with_memory.invoke(
            {
                "messages": [
                    {"role": "user", "content": user_input}
                ]
            },
            config={"configurable": {"session_id": session_id}}
        )

        messages = result["messages"]

        # =====================
        # ✅ 打印工具调用过程
        # =====================
        print("\n🧠【Agent执行过程】")

        for msg in messages:
            # ===== AI 决定调用工具 =====
            if isinstance(msg, AIMessage) and msg.tool_calls:
                for call in msg.tool_calls:
                    print("\n🔧 AI 决定调用工具")
                    print("工具名：", call["name"])
                    print("参数：", call["args"])

            # ===== 工具返回结果 =====
            if isinstance(msg, ToolMessage):
                print("\n📦 工具执行结果")
                print(msg.content)

        # =====================
        # ✅ 最终回答
        # =====================
        print("\n✅【最终回答】")
        print(messages[-1].content)
        print()


===== 🧠 LangChain v1.2 Agent =====

👤【用户输入】

